In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
from data.pascalvoc.preprocessing.utils import load_ids
from data.pascalvoc.pascalvoc import PascalVOC

Using TensorFlow backend.


In [4]:
from model.networks.baseline_logits import BaselineLogits

In [5]:
from model import relabel

In [6]:
data_dir = '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/'
weights_path = '/home/caleml/partial_experiments/exp_20191009_1546_baseline_logits/weights_10_015_00.h5'
gt_path = os.path.join(data_dir, 'Annotations', 'annotations_multilabel_trainval_partial_100_1.csv')
config_path = '/home/caleml/partial-labels/config/pv_relabel_pfull.yaml'
config_path2 = '/home/caleml/partial-labels/config/pv_relabel_pcoco_alpha.yaml'

In [7]:
batch_size = 16
nb_classes = 20
prop = 10

In [8]:
class_info = load_ids()
id2name = {int(info['id']): info['name'] for info in class_info.values()}

In [9]:
# load model
model = BaselineLogits('%s/partial_experiments/' % os.environ['HOME'], nb_classes, prop)
model.load_weights(weights_path)
model.load_config(config_path2)

Init input_shape (448, 448, 3)
Loading options
{'ARCHI': {'CLASSIFIER': 'resnet50', 'LOSS': 'bce', 'NAME': 'baseline_logits'},
 'BATCH_SIZE': 16,
 'CALLBACK': {'LR_FACTOR': 0.1,
              'LR_TRIGGER': [],
              'MIN_DELTA': 0.0,
              'PATIENCE': 10,
              'PATIENCE_LR': 2,
              'TENSORBOARD': {'SAVE_GRAPH': True, 'USE_TENSORBOARD': False},
              'VAL_CB': 'map'},
 'CLEANUP': False,
 'DATAAUGMENTATION': {'DEFAULT_DICT': {'HORIZONTAL_FLIP': True,
                                       'horizontal_flip': True}},
 'DATASET': {'NAME': 'pascalvoc',
             'PATH': '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/',
             'SHUFFLE': True,
             'TEST': 'test',
             'TRAIN': 'trainval'},
 'DEBUG': {'IS_TEST': False},
 'EPSILON': 1e-07,
 'IMAGE': {'IMG_SIZE': 448, 'N_CHANNELS': 3},
 'MULTIP': {'MAX_QUEUE_SIZE': 10, 'N_WORKERS': 2, 'USE_MULTIPROCESS': False},
 'RANDOM_SEED': 1,
 'RELABEL': {'ACTIVE': True,
             '

/home/caleml/partial-labels/model/networks/__init__.py:39: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  loaded_conf = yaml.load(f_in)


Outputs shape [(None, 20), (None, 20)]
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
reduction_indices is deprecated, use axis instead
Final model summary
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, 448, 448, 3) 0                                            
__________________________________________________________________________________________________
cls_model (Model)               (None, 14, 14, 2048) 23587712    image_input[0][0]                
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 2048)         0           cls_model[1][0]                  
______________________________________________________________

In [10]:
# load dataset
dataset_train = PascalVOC(data_dir, batch_size, 'trainval', x_keys=['image', 'image_id'], y_keys=['multilabel'], p=prop)

In [13]:
# init relabelator
options = {
    'type': 'proportion_yf',
    'prop_pos': 0.044,
    'prop_neg': 0.78,
    'prior': 'conditional_coco',
    'prior_path': '/home/caleml/datasets/mscoco/annotations/multilabel_train2014_partial_100_1.json',
    'prior_combination': 'alpha',
    'prior_alpha': 0.45
}
relabelator = relabel.PriorRelabeling('/home/caleml/partial-labels/experiments/tests', prop, nb_classes, options)

loading conditional prior with prior_type coco and method alpha


In [12]:
# loop on the train dataset to create the relabeling dataset
relabel_step = 0
relabelator.init_step(relabel_step)

# predict
for i in range(len(dataset_train)):
    if i % 100 == 0:
        print('doing batch %s' % i)
        
    x_batch, y_batch = dataset_train[i]

    y_pred = model.predict(x_batch)   # TODO not the logits!!!!!!!!

    relabelator.relabel(x_batch, y_batch, y_pred)

relabelator.finish_step(relabel_step)
targets_path = relabelator.targets_path

print('new relabel path is %s' % targets_path)

doing batch 0
doing batch 100
doing batch 200
doing batch 300
	Added 73485 labels during relabeling, logging into /home/caleml/partial-labels/experiments/tests/relabeling/log_relabeling.csv
	New dataset path /home/caleml/partial-labels/experiments/tests/relabeling/relabeling_0_10p.csv
new relabel path is /home/caleml/partial-labels/experiments/tests/relabeling/relabeling_0_10p.csv


In [ ]:
# evaluate the relabeled dataset